In [1]:
import os
import pandas as pd
import numpy as np
import re

In [2]:
def rox_label():
    Gene_list = []
    miRNA_list = []
    
    DNA_list = ['A','T','G','C']
    RNA_list = ['A','U','G','C']
    
    for a in DNA_list:
        for b in DNA_list:
            for c in DNA_list:
                        Gene_list.append(a+b+c)
                    
    for a in RNA_list:
        for b in RNA_list:
            for c in RNA_list:
                        miRNA_list.append(a+b+c)
                        
    return(Gene_list,miRNA_list)

In [3]:
Gene_list = rox_label()[0]

In [4]:
miRNA_list = rox_label()[1]

In [5]:
FILE_PATH_gene = '/Users/kongxuwen/Desktop/竞赛/train_dataset/gene_seq.csv'
FILE_PATH_miRNA = '/Users/kongxuwen/Desktop/竞赛/train_dataset/mirna_seq.csv'

In [6]:
gene_data = pd.read_csv(FILE_PATH_gene)
print(gene_data)

        label                                           sequence
0        A1BG  TGCAGCCGCGGGCCCAGGGTGCTGTTGGTGTCCTCAGAAGTGCCGG...
1        A1CF  AGATGCTTTTTTAAATTTAAGAATAAGACACACAAAACTCTATTAA...
2       A2BP1  CCTGCCCGCGAAGTTGCGGACAGTGCGTGAGAAACCAGCACCCCCT...
3       A2LD1  GGGGGACGGGCAGGGTGGGCCTAGGTTTGAGAGCCCTGGGGCTCCA...
4         A2M  ATGACCAGGACAATGAAGACTGCATCAATCGTCATAATGTCTATAT...
...       ...                                                ...
16122  ZYG11A  CAGGTTTGATGCTGCCAAGTTTGTCATGAGATGGCTCTGTAAGCAT...
16123  ZYG11B  TAGCCATAAGTATTGGATAGTTGAATCACAGGAATCCTTTTTGTGA...
16124     ZYX  AGGCATGGAATAGGTGCTCTGACCTCTGACCCTCTAGCCCAGGGAG...
16125   ZZEF1  CTCCTTTTTGATCCTGTCCCTCACCCACCTTCACACACACACACGC...
16126    ZZZ3  GGCCAGGTCCAGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGG...

[16127 rows x 2 columns]


In [7]:
rna_data = pd.read_csv(FILE_PATH_miRNA)
print(rna_data)

                mirna                       seq
0       hsa-let-7a-5p  UGAGGUAGUAGGUUGUAUAGUU\n
1       hsa-let-7a-3p   CUAUACAAUCUACUGUCUUUC\n
2     hsa-let-7a-2-3p  CUGUACAGCCUCCUAGCUUUCC\n
3       hsa-let-7b-5p  UGAGGUAGUAGGUUGUGUGGUU\n
4       hsa-let-7b-3p  CUAUACAACCUACUGCCUUCCC\n
...               ...                       ...
2651    hsa-miR-12131    UUUGGAGAGGUGUACUCCCA\n
2652    hsa-miR-12132  UAUUACUGUGAGAAUUAUGAUG\n
2653    hsa-miR-12133  CUUGGCACCAUUAAAAAGUACA\n
2654    hsa-miR-12135      UAAAGGUUUGUUUGUAAA\n
2655    hsa-miR-12136      GAAAAAGUCAUGGAGGCC\n

[2656 rows x 2 columns]


In [8]:
def Data(gene_data,Gene_list,rna_data,miRNA_list):
    gene_matrtix = np.empty([16127,len(Gene_list)],dtype = int)
    
    for i in range(len(Gene_list)):
        for j in range(0,16127):
            Seq = gene_data.loc[j,'sequence']
            pattern = re.compile(Gene_list[i])

            Match_list = pattern.findall(Seq)
            gene_matrtix[j][i] = len(Match_list)
            
    rna_matrtix = np.empty([2656,len(miRNA_list)],dtype = int)
    
    for i in range(len(miRNA_list)):
        for j in range(0,2656):
            Seq = rna_data.loc[j,'seq']
            pattern = re.compile(miRNA_list[i])

            Match_list = pattern.findall(Seq)
            rna_matrtix[j][i] = len(Match_list)
            
    return(gene_matrtix,rna_matrtix)

In [9]:
gene_matrtix,rna_matrtix = Data(gene_data,Gene_list,rna_data,miRNA_list)

In [10]:
FILE_PATH_Train = '/Users/kongxuwen/Desktop/竞赛/train_dataset/Train.csv'

In [11]:
def Data_Fix(FILE_PATH):
    data = pd.read_csv(FILE_PATH)
    
    Data = np.empty([len(data),len(Gene_list)+len(miRNA_list)+1],dtype = int)
    
    if(data.shape[1]==3):
        D = {'Functional MTI':1,'Non-Functional MTI':0}

        for i in range(len(data)):
            data.loc[i,'label'] = D[data.loc[i,'label']]
            
        for i in range(len(data)):
            Data[i][-1] = data.loc[i,'label']
                   
    for i in range(len(data)):
        index_gene =  gene_data[gene_data['label']==data.loc[i,'gene']].index.tolist()
        for j in range(len(Gene_list)-1):
            Data[i][j] = gene_matrtix[index_gene[0]][j]

        index_rna =  rna_data[rna_data['mirna']==data.loc[i,'miRNA']].index.tolist()
        for j in range(len(miRNA_list)-1):
            Data[i][j+len(Gene_list)] = rna_matrtix[index_rna[0]][j]
            
    return(Data)

In [12]:
Train_data = Data_Fix(FILE_PATH_Train)

FILE_PATH_test = '/Users/kongxuwen/Desktop/竞赛/test_dataset.csv'

Test_data = Data_Fix(FILE_PATH_test)

In [13]:
Train_data.shape

(738, 129)

In [14]:
Test_data.shape

(185, 129)

In [15]:
X = np.vstack((Train_data,Test_data))

In [16]:
X.shape

(923, 129)

In [17]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [18]:
X_scaler = StandardScaler()
x = X_scaler.fit_transform(X[:,:-1])

In [19]:
x.shape

(923, 128)

In [20]:
pca = PCA(n_components=0.99)
pca.fit(x)
x_pca = pca.transform(x)

In [21]:
x_pca.shape

(923, 74)

In [22]:
len(x_pca[0][:])

74

In [23]:
import torch
from torch import nn
import torch.nn.functional as F

In [24]:
n_features = len(x_pca[0][:])

features = torch.from_numpy(x_pca[:738,:]).to(torch.float32)

labels = torch.from_numpy(Train_data[:,-1]).to(torch.float32)

In [25]:
features.shape

torch.Size([738, 74])

In [26]:
class LogisticRegressionModel(nn.Module):
    def __init__(self,n_features):
        super(LogisticRegressionModel,self).__init__()
        self.linear = nn.Linear(n_features,1)
        self.Sigmid = nn.Sigmoid()
        
    def forward(self,x):
        y_pred = torch.sigmoid(self.linear(x))
        y_pred = y_pred.squeeze(-1)
        return y_pred

In [27]:
from torch.nn import init

In [28]:
import torch.utils.data as Data

In [29]:
batch_size = 20
dataset = Data.TensorDataset(features,labels)
data_iter = Data.DataLoader(dataset,batch_size,shuffle=True)

In [30]:
n_hidden = 50

net = nn.Sequential(
    nn.Linear(n_features,n_hidden),
    nn.ReLU(),
    LogisticRegressionModel(n_hidden)
)

for params in net.parameters():
    init.normal_(params, mean=0, std=0.01)

In [31]:
from torch.nn import init

In [32]:
criterion = torch.nn.BCEWithLogitsLoss()

optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

In [33]:
for epoch in range(10000):
    for x,y in data_iter:
        y_pred = net(x)
#         print(y_pred)
        loss = criterion(y_pred,y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(epoch,loss.item())
#     for name, parms in model.named_parameters():
#     print('-->name:',name,'-->grad_requirs:',parms.requires_grad,'--weight',torch.mean(parms.data),'-->grad_value:',torch.mean(parms.grad))

0 0.6126506328582764
1 0.5843521952629089
2 0.6404455900192261
3 0.6968794465065002
4 0.6118506789207458
5 0.6973010897636414
6 0.5259204506874084
7 0.7546823024749756
8 0.668694794178009
9 0.6397531032562256
10 0.5238960385322571
11 0.5814189314842224
12 0.6396331191062927
13 0.639639139175415
14 0.6396397352218628
15 0.5808079838752747
16 0.6686431765556335
17 0.5501587390899658
18 0.6095514893531799
19 0.6093999147415161
20 0.638669490814209
21 0.6386914253234863
22 0.5787938237190247
23 0.6389538645744324
24 0.6691149473190308
25 0.7002028226852417
26 0.6077383756637573
27 0.6084573268890381
28 0.5771573781967163
29 0.6697332859039307
30 0.6070404648780823
31 0.5144324898719788
32 0.5444463491439819
33 0.5757415294647217
34 0.6701284050941467
35 0.6070646047592163
36 0.5751597881317139
37 0.606453537940979
38 0.7003929018974304
39 0.6377777457237244
40 0.47822630405426025
41 0.7326294183731079
42 0.6386687159538269
43 0.6392617225646973
44 0.5090988278388977
45 0.5415709614753723
4

373 0.5935898423194885
374 0.6105878353118896
375 0.5080011487007141
376 0.5055645704269409
377 0.5089944005012512
378 0.5395168662071228
379 0.6633105874061584
380 0.5083441734313965
381 0.5588805079460144
382 0.6724846363067627
383 0.4631587564945221
384 0.6006309986114502
385 0.5452418327331543
386 0.6045032739639282
387 0.5331733226776123
388 0.6768946647644043
389 0.5025121569633484
390 0.5537824034690857
391 0.5530771613121033
392 0.5054572820663452
393 0.7103804349899292
394 0.516175389289856
395 0.4884747862815857
396 0.46595996618270874
397 0.5712988972663879
398 0.6342642307281494
399 0.6163367033004761
400 0.6416518688201904
401 0.5609598755836487
402 0.47406721115112305
403 0.5535534024238586
404 0.5147547721862793
405 0.6652284264564514
406 0.5688116550445557
407 0.4409646689891815
408 0.6831343770027161
409 0.5541940927505493
410 0.5936616659164429
411 0.4353092908859253
412 0.5228626728057861
413 0.6194745302200317
414 0.5147207975387573
415 0.5760714411735535
416 0.5709

737 0.5683730840682983
738 0.5284364223480225
739 0.5086437463760376
740 0.44571051001548767
741 0.46545374393463135
742 0.6186457872390747
743 0.491449236869812
744 0.5749150514602661
745 0.6532207727432251
746 0.4491414427757263
747 0.5302484631538391
748 0.5407117605209351
749 0.47436776757240295
750 0.3706459403038025
751 0.4777027666568756
752 0.4355529844760895
753 0.4960015118122101
754 0.5641542077064514
755 0.49071863293647766
756 0.5861122608184814
757 0.5364910960197449
758 0.5648481249809265
759 0.45482906699180603
760 0.44315826892852783
761 0.6852355003356934
762 0.6087696552276611
763 0.5344455242156982
764 0.5401418805122375
765 0.4695795774459839
766 0.4687598645687103
767 0.4593789875507355
768 0.4154188632965088
769 0.6251463890075684
770 0.49985164403915405
771 0.3915160894393921
772 0.4159349501132965
773 0.6120778918266296
774 0.5046520829200745
775 0.5732686519622803
776 0.4665471613407135
777 0.48481008410453796
778 0.43025174736976624
779 0.4974455237388611
780

1098 0.45722696185112
1099 0.3916413486003876
1100 0.48339325189590454
1101 0.49770763516426086
1102 0.48288536071777344
1103 0.5055825710296631
1104 0.4578811824321747
1105 0.47649383544921875
1106 0.5082593560218811
1107 0.5463981032371521
1108 0.4004404544830322
1109 0.5137158036231995
1110 0.5659571290016174
1111 0.435735821723938
1112 0.5728998780250549
1113 0.5251486897468567
1114 0.48920685052871704
1115 0.5192816853523254
1116 0.4450863301753998
1117 0.5927923917770386
1118 0.5188648700714111
1119 0.4387301206588745
1120 0.4563228189945221
1121 0.49578142166137695
1122 0.43450239300727844
1123 0.48986202478408813
1124 0.4307246208190918
1125 0.5862076282501221
1126 0.5098714828491211
1127 0.5226574540138245
1128 0.4814862608909607
1129 0.40023908019065857
1130 0.5084483623504639
1131 0.4427262842655182
1132 0.5015866756439209
1133 0.3866282105445862
1134 0.5048509836196899
1135 0.5468471646308899
1136 0.45220062136650085
1137 0.4998835027217865
1138 0.47145605087280273
1139 0.4

1439 0.6346259713172913
1440 0.4338090717792511
1441 0.5370492935180664
1442 0.4028225541114807
1443 0.45950350165367126
1444 0.47361087799072266
1445 0.5656524896621704
1446 0.3655591905117035
1447 0.5449679493904114
1448 0.44603675603866577
1449 0.6373607516288757
1450 0.534379243850708
1451 0.4628748893737793
1452 0.3913310766220093
1453 0.6254854202270508
1454 0.4597417414188385
1455 0.546646237373352
1456 0.5608132481575012
1457 0.4878997206687927
1458 0.44199129939079285
1459 0.4381953477859497
1460 0.5826030969619751
1461 0.5295314788818359
1462 0.5544840097427368
1463 0.43879833817481995
1464 0.4717773199081421
1465 0.5631672739982605
1466 0.4800621569156647
1467 0.5365259051322937
1468 0.4366726875305176
1469 0.5087369680404663
1470 0.41264045238494873
1471 0.3893268406391144
1472 0.4012399911880493
1473 0.4717523753643036
1474 0.48070716857910156
1475 0.5473172664642334
1476 0.48246151208877563
1477 0.4373905658721924
1478 0.4612962007522583
1479 0.3859909772872925
1480 0.420

...

8620 0.4412463903427124
8621 0.4335668683052063
8622 0.4190255105495453
8623 0.4401721656322479
8624 0.4538525342941284
8625 0.37699660658836365
8626 0.4756354093551636
8627 0.45342862606048584
8628 0.4829188585281372
8629 0.39857926964759827
8630 0.5095803737640381
8631 0.5170013904571533
8632 0.3768690824508667
8633 0.5380266308784485
8634 0.3768921494483948
8635 0.46692705154418945
8636 0.4401135742664337
8637 0.41936975717544556
8638 0.4823331832885742
8639 0.4321661591529846
8640 0.538921594619751
8641 0.5230578184127808
8642 0.4536375403404236
8643 0.49555307626724243
8644 0.390543669462204
8645 0.4199579954147339
8646 0.47437870502471924
8647 0.5510690808296204
8648 0.5302934050559998
8649 0.44079774618148804
8650 0.4750385880470276
8651 0.5183058977127075
8652 0.5104005336761475
8653 0.4614294469356537
8654 0.4189335107803345
8655 0.4533044099807739
8656 0.5092989802360535
8657 0.4190104305744171
8658 0.44036197662353516
8659 0.39915820956230164
8660 0.398053377866745
8661 0.48

8962 0.4540793001651764
8963 0.46290192008018494
8964 0.41890445351600647
8965 0.5089850425720215
8966 0.5390169620513916
8967 0.4879990816116333
8968 0.482974648475647
8969 0.5167964100837708
8970 0.46156951785087585
8971 0.45344415307044983
8972 0.4882816672325134
8973 0.5089094638824463
8974 0.5519022345542908
8975 0.4955642521381378
8976 0.5168503522872925
8977 0.4957716166973114
8978 0.397786021232605
8979 0.47496864199638367
8980 0.4610951244831085
8981 0.3981243371963501
8982 0.4962315559387207
8983 0.4880780577659607
8984 0.5593636631965637
8985 0.3980944752693176
8986 0.4961589276790619
8987 0.4877112805843353
8988 0.49587202072143555
8989 0.47459399700164795
8990 0.47438859939575195
8991 0.474791944026947
8992 0.4199354648590088
8993 0.49566444754600525
8994 0.4827752113342285
8995 0.4744437634944916
8996 0.33448436856269836
8997 0.5733678340911865
8998 0.4745784401893616
8999 0.47497162222862244
9000 0.355734258890152
9001 0.4193492531776428
9002 0.4323165714740753
9003 0.44

9301 0.5377795696258545
9302 0.48237621784210205
9303 0.4190860986709595
9304 0.3767656087875366
9305 0.47566044330596924
9306 0.43241041898727417
9307 0.5378822088241577
9308 0.5193467736244202
9309 0.45394235849380493
9310 0.39848095178604126
9311 0.4746032953262329
9312 0.46137720346450806
9313 0.5384753346443176
9314 0.5728789567947388
9315 0.39827924966812134
9316 0.3979092836380005
9317 0.46226751804351807
9318 0.4195561110973358
9319 0.5378753542900085
9320 0.45506441593170166
9321 0.4836907386779785
9322 0.3985907733440399
9323 0.4920464754104614
9324 0.39780810475349426
9325 0.3913165330886841
9326 0.49616390466690063
9327 0.48780739307403564
9328 0.4407229721546173
9329 0.5091065168380737
9330 0.39800527691841125
9331 0.5301526188850403
9332 0.4671171009540558
9333 0.5377855896949768
9334 0.47447019815444946
9335 0.5041887760162354
9336 0.39876604080200195
9337 0.49576202034950256
9338 0.3980739116668701
9339 0.5723040103912354
9340 0.45336464047431946
9341 0.5049338936805725

9647 0.39895105361938477
9648 0.5299117565155029
9649 0.45630401372909546
9650 0.4403376281261444
9651 0.47470319271087646
9652 0.5299370288848877
9653 0.4548828899860382
9654 0.5936307311058044
9655 0.43229737877845764
9656 0.37679818272590637
9657 0.5181298851966858
9658 0.48825740814208984
9659 0.35592302680015564
9660 0.5168784260749817
9661 0.4328906238079071
9662 0.4192700684070587
9663 0.5855205655097961
9664 0.4955407679080963
9665 0.4191815257072449
9666 0.47478294372558594
9667 0.4613422751426697
9668 0.49595844745635986
9669 0.41915762424468994
9670 0.3766702711582184
9671 0.46167272329330444
9672 0.495929092168808
9673 0.5168254375457764
9674 0.4404138922691345
9675 0.419597327709198
9676 0.4339095950126648
9677 0.44120320677757263
9678 0.505691409111023
9679 0.5654167532920837
9680 0.47551992535591125
9681 0.41917216777801514
9682 0.4828322231769562
9683 0.41923701763153076
9684 0.4189356565475464
9685 0.4189951717853546
9686 0.3771381676197052
9687 0.41978392004966736
968

9989 0.411966472864151
9990 0.4613131880760193
9991 0.5414555668830872
9992 0.47505345940589905
9993 0.5098331570625305
9994 0.44030824303627014
9995 0.4858251214027405
9996 0.41931116580963135
9997 0.3771240711212158
9998 0.3346966803073883
9999 0.3978138566017151


In [34]:
features_test = torch.from_numpy(x_pca[738:923,:]).to(torch.float32)

In [35]:
net(features_test).detach().numpy()

array([9.97474849e-01, 7.29530996e-07, 1.00000000e+00, 9.70043242e-01,
       8.73971312e-11, 1.00000000e+00, 1.00000000e+00, 6.45356327e-02,
       1.00000000e+00, 1.00000000e+00, 1.35260075e-02, 1.00000000e+00,
       9.99973178e-01, 1.00000000e+00, 6.17136294e-03, 3.58781725e-01,
       9.97630477e-01, 1.00000000e+00, 9.89993393e-01, 9.99997735e-01,
       6.04868829e-01, 1.09982147e-05, 1.00000000e+00, 1.00000000e+00,
       2.41653454e-02, 1.00000000e+00, 1.00000000e+00, 1.53910369e-01,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 6.18230343e-01,
       9.97161388e-01, 4.27606851e-01, 6.39812768e-01, 9.99979138e-01,
       1.00000000e+00, 1.00000000e+00, 9.98761535e-01, 1.00000000e+00,
       9.99998689e-01, 1.00000000e+00, 1.00000000e+00, 8.60234124e-08,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       9.65532839e-01, 3.60940024e-02, 1.00000000e+00, 3.65231926e-07,
       1.00000000e+00, 1.00000000e+00, 2.77996361e-02, 1.00000000e+00,
      

In [36]:
result = pd.DataFrame(net(features_test).detach().numpy(),dtype=float)

In [37]:
result

,0
0,9.974748e-01
1,7.295310e-07
2,1.000000e+00
3,9.700432e-01
4,8.739713e-11
...,...
180,9.997582e-01
181,1.000000e+00
182,1.000000e+00
183,1.000000e+00


In [38]:
test = pd.read_csv(FILE_PATH_test)

In [39]:
outfile = pd.concat([test,result],axis=1)

In [40]:
outfile

,gene,miRNA,0
0,PTBP2,hsa-miR-133b,9.974748e-01
1,ITGB8,hsa-miR-19b-1-5p,7.295310e-07
2,RB1,hsa-miR-132-3p,1.000000e+00
3,CSNK2A1,hsa-miR-760,9.700432e-01
4,ACVR1C,hsa-miR-376b-3p,8.739713e-11
...,...,...,...
180,SNX19,hsa-miR-376a-5p,9.997582e-01
181,IGF1R,hsa-miR-125b-2-3p,1.000000e+00
182,XPA,hsa-miR-373-3p,1.000000e+00
183,MEGF9,hsa-miR-125b-5p,1.000000e+00


In [41]:
outfile.to_csv('/Users/kongxuwen/Desktop/result01.csv')